# PD 7 Szymon Rećko

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics import precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

## Dane

In [ ]:
train = pd.read_csv('train.csv')
val = pd.read_csv('val.csv')
test = pd.read_csv('test.csv')

In [ ]:
train

In [ ]:
test

## Model
Wiemy że na zbiorze treningowym nie ma outliarów. Zaczynamy od wytrenowania na zbiorze treningowym poszukując dobrej liczby komponentów minimalizując BIC.

In [ ]:
bic = []
for i in range(1, 10):
    gmm_temp = GaussianMixture(n_components = i, random_state = 42, covariance_type = "full")
    gmm_temp.fit(train)
    bic.append(gmm_temp.bic(train))

In [ ]:
bic

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, 10), bic)
plt.xlabel("Liczba komponentów")
plt.ylabel("BIC")
plt.show()

Minimum jest dla jednego komponentu więc to przy nim zostaniemy.

In [ ]:
gmm = GaussianMixture(n_components=1, random_state=123)
gmm.fit(train)

Wybieramy treshold porównując prozkłady przynależenia do klastra w zbiorze treningowym i walidacyjnym.

In [ ]:
inliers_probability = gmm.score_samples(train)
pd.DataFrame(inliers_probability).hist()

In [ ]:
all_probability = gmm.score_samples(val)
pd.DataFrame(all_probability).hist()

Ustawiamy treshold na -30

## Metryki

In [ ]:
test_probability = gmm.score_samples(test.drop(columns=["class"]))
classified = (test_probability < -30).astype(int)
y_real = test["class"]

In [ ]:
results = pd.DataFrame(
    [{"Metric":"precision", "Value": precision_score(y_real, classified)},
     {"Metric":"recall", "Value": recall_score(y_real, classified)},
     {"Metric":"score", "Value": f1_score(y_real, classified)}],
    columns=["Metric", "Value"]
)
results

Na podanym zbiorze danych z udało się za pomocą GMM stworzyć klasyfikator wartości odstających.